In [1]:
import pandas as pd
import mido
from zipfile import ZipFile

In [2]:
def midi2nmat(path):
    """Returns a Note Matrix from a given MIDI files"""
    
    assert (type(path) is str), "Filepath must be a string: %r" % path
    mid = mido.MidiFile(path)
    tpb = mid.ticks_per_beat

    midiframe = pd.DataFrame(columns=["Type", "Voice", "Pitch", "Velocity", "Ticks"])

    for track in mid.tracks:
        for msg in track:
            if msg.type == "note_on":
                df = pd.DataFrame({"Type":msg.type, "Voice":msg.channel, "Pitch":msg.note, "Velocity":msg.velocity, "Ticks":msg.time},\
                              index=[0])
            else:
                df = pd.DataFrame({"Type":msg.type, "Ticks":msg.time},\
                              index=[0])
            midiframe = midiframe.append(df, ignore_index=True)

    midiframe["Time"] = pd.Series(midiframe["Ticks"].cumsum() / tpb, index=midiframe.index)

    note_on = midiframe.loc[(midiframe["Velocity"] != 0) & (midiframe["Type"] == "note_on")]
    note_off = midiframe.loc[(midiframe["Velocity"] == 0) & (midiframe["Type"] == "note_on")]
    newdex = range(0,len(note_off))

    note_off = note_off.reset_index(drop=True)
    note_on = note_on.reset_index(drop=True)

    note_on["Duration"] = pd.Series(note_off["Time"] - note_on["Time"], index = note_on.index)

    return note_on[["Pitch", "Time", "Velocity", "Voice", "Duration"]]

In [3]:
def _nmatPloy2Mono(nmat):
    """Turns Polyphonic MIDI File into Monophonic MIDI file"""
    #OBSOLETE AS OF 
    #add voice * onset of the last beat in voice 0 to onsets for v1 and v2

In [12]:
def _unzipmxl(path):
    with ZipFile(path) as zipObj:
        listOfFileNames = zipObj.namelist()
        zipObj.extractall("../Data")
        print(listOfFileNames)
    return("../Data/"+listOfFileNames[2])

In [13]:
path1 = "../Data/Fugue-BWV-847.mid"
path2 = "../Data/a-exsultate.mid"
zipPath = "../Data/OMOLUABI XML.mxl"
xmlPath = _unzipmxl(zipPath)
nmat = midi2nmat(path2)

['META-INF/', 'META-INF/container.xml', 'OMOLUABI XML.xml']


In [14]:
xmlPath

'../Data/OMOLUABI XML.xml'